In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import re
import tensorflow as tf
import string
import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [ ]:
client = bigquery.Client()
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


In [ ]:
sql = """SELECT id, title , tags 
        FROM bigquery-public-data.stackoverflow.stackoverflow_posts
        WHERE title NOT LIKE '%None%'
        LIMIT 10000
        """

results = client.query(sql).to_dataframe()

results.head()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,id,title,tags
0,33142013,not able to get reference to MessageEventManag...,android|xmpp|smack|status|typing
1,36356000,docker registry v2 upload fails for some images,nginx|docker|docker-registry|wercker
2,22884909,URL Scheme to open specific View with data,ios|objective-c|xcode|url-scheme
3,31844558,Group chat using jabber IMCore API,xmpp|multiuserchat
4,27920190,accessing a map from Cassandra in R,r|serialization|dictionary|cassandra|key-value


In [ ]:
tags = []
for tag in results['tags']:
    tag = tag.split('|')
    if len(tag)<20:
        tags.append(tag)
        
df = pd.DataFrame({'id': results['id'], 'text': results['title'], 'tags' : tags})
df.head()

,id,text,tags
0,33142013,not able to get reference to MessageEventManag...,"[android, xmpp, smack, status, typing]"
1,36356000,docker registry v2 upload fails for some images,"[nginx, docker, docker-registry, wercker]"
2,22884909,URL Scheme to open specific View with data,"[ios, objective-c, xcode, url-scheme]"
3,31844558,Group chat using jabber IMCore API,"[xmpp, multiuserchat]"
4,27920190,accessing a map from Cassandra in R,"[r, serialization, dictionary, cassandra, key-..."


In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

punctuations = list(string.punctuation)
punctuations.remove("#")
punctuations.remove("+")

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)
def remove_punctuations(text):
    no_punctuations_text = [w for w in text.split() if not w in punctuations]
    return ' '.join(no_punctuations_text)

In [ ]:
def clean_text(text):
    text = text.lower()
    text = remove_punctuations(text)
    text = remove_stopwords(text)
    return text

In [ ]:
df['clean_text'] = df['text'].apply(lambda x: clean_text(x))
df.head()

,id,text,tags,clean_text
0,33142013,not able to get reference to MessageEventManag...,"[android, xmpp, smack, status, typing]",able get reference messageeventmanager class s...
1,36356000,docker registry v2 upload fails for some images,"[nginx, docker, docker-registry, wercker]",docker registry v2 upload fails images
2,22884909,URL Scheme to open specific View with data,"[ios, objective-c, xcode, url-scheme]",url scheme open specific view data
3,31844558,Group chat using jabber IMCore API,"[xmpp, multiuserchat]",group chat using jabber imcore api
4,27920190,accessing a map from Cassandra in R,"[r, serialization, dictionary, cassandra, key-...",accessing map cassandra r


In [ ]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df['tags'])

y = multilabel_binarizer.transform(df['tags'])

y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
train, test, y_train, y_test = train_test_split(df, y, test_size=0.2)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
train.shape, test.shape

((8000, 4), (2000, 4))

In [ ]:
train_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<OOV>')
train_tokenizer.fit_on_texts(train['clean_text'].values)
train_word_index = train_tokenizer.word_index

In [ ]:
vocab_length = len(train_word_index) + 1
vocab_length

9168

In [ ]:
train_sequences = train_tokenizer.texts_to_sequences(train['clean_text'].values)
len(train_sequences)

8000

In [ ]:
test_sequences = train_tokenizer.texts_to_sequences(test['clean_text'].values)
len(test_sequences)

2000

In [ ]:
longest_sentence = len(max(train_sequences, key=len))
train_padded_seqeunces = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, padding='post', maxlen=longest_sentence, truncating='post')
test_padded_seqeunces = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, padding='post', maxlen=longest_sentence, truncating='post')
longest_sentence

23

In [ ]:
tf.keras.backend.clear_session()

embedding_dim = 50
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_length, embedding_dim, input_length=longest_sentence),
    
    tf.keras.layers.LSTM(500, return_sequences=True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(250, return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LSTM(100, return_sequences=True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    
    tf.keras.layers.Dense(y.shape[1], activation='softmax')
])

2022-10-18 17:25:49.127921: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 23, 50)            458400    
_________________________________________________________________
lstm (LSTM)                  (None, 23, 500)           1102000   
_________________________________________________________________
batch_normalization (BatchNo (None, 23, 500)           2000      
_________________________________________________________________
dropout (Dropout)            (None, 23, 500)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 23, 250)           751000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 23, 250)           0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 23, 250)           1

In [ ]:
history = model.fit(train_padded_seqeunces, y_train, epochs=10, 
                    validation_data=(test_padded_seqeunces, y_test))

2022-10-18 17:26:06.426895: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
250/250 [==============================] - 59s 213ms/step - loss: 22.8858 - accuracy: 0.0362 - val_loss: 22.1003 - val_accuracy: 0.0175
Epoch 2/10
250/250 [==============================] - 53s 211ms/step - loss: 86.2562 - accuracy: 0.0316 - val_loss: 62.3543 - val_accuracy: 0.0445
Epoch 3/10
250/250 [==============================] - 53s 212ms/step - loss: 755.3975 - accuracy: 0.0205 - val_loss: 2496.1384 - val_accuracy: 0.0095
Epoch 4/10
250/250 [==============================] - 54s 214ms/step - loss: 2357.7878 - accuracy: 0.0156 - val_loss: 5170.4238 - val_accuracy: 0.0180
Epoch 5/10
250/250 [==============================] - 53s 212ms/step - loss: 4585.0132 - accuracy: 0.0150 - val_loss: 7649.1655 - val_accuracy: 0.0060
Epoch 6/10
250/250 [==============================] - 53s 213ms/step - loss: 7411.6787 - accuracy: 0.0139 - val_loss: 8401.3145 - val_accuracy: 0.0430
Epoch 7/10
250/250 [==============================] - 53s 212ms/step - loss: 10805.4814 - accuracy: 0.0

In [ ]:
test_prob = model.predict(test_padded_seqeunces)
t = 0.2
test_pred = (test_prob >= t).astype(int)

In [ ]:
for i in range(10):
    k = test.sample(1).index[0]
    print("Title: ", test['text'][k])
    print("Actual Tag: ", multilabel_binarizer.inverse_transform(y_test[k].reshape(1, -1))[0])
    print("Predicted tag: ", multilabel_binarizer.inverse_transform(test_pred[k].reshape(1, -1))[0])
    print('\n')

Title:  ActiveMQ mutual SSL authentication
Actual Tag:  ('activemq', 'authentication', 'configuration', 'mutual', 'ssl')
Predicted tag:  ('django',)


Title:  Youtube Android API feature similar to Spotify's Media Notifications?
Actual Tag:  ('android', 'ios', 'youtube')
Predicted tag:  ('django', 'python')


Title:  How to serialize SqlAlchemy join query to JSON?
Actual Tag:  ('flask-sqlalchemy', 'json', 'python', 'sqlalchemy')
Predicted tag:  ('django',)


Title:  how to see jsp page in eclipse while designing?
Actual Tag:  ('eclipse',)
Predicted tag:  ('django',)


Title:  Crystal Report Viewer Not Accepting Login Info
Actual Tag:  ('c#', 'crystal-reports', 'winforms')
Predicted tag:  ('django', 'ios', 'python')


Title:  How to cancel a long time operation in WxPerl
Actual Tag:  ('perl', 'wx', 'wxperl', 'wxwidgets')
Predicted tag:  ('django', 'python')


Title:  How to download a file with wget and save it according to the http-reported filename?
Actual Tag:  ('bash', 'http-headers